In [42]:
import transformers
import torch
BERT_MODEL_PATH = r"F:/workcode/FAQ/BERT_model"
# a.通过词典导入分词器
tokenizer = transformers.BertTokenizer.from_pretrained(BERT_MODEL_PATH+'/bert-base-uncased-vocab.txt') 
# b. 导入配置文件
model_config = transformers.BertConfig.from_pretrained(BERT_MODEL_PATH+'/bert-base-uncased-config.json')
# 修改配置
#model_config.output_hidden_states = True
#model_config.output_attentions = True
# 通过配置和路径导入模型
model = transformers.BertModel.from_pretrained(BERT_MODEL_PATH+'/bert-base-uncased-pytorch_model.bin',config = model_config)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [43]:
# encode_plus返回所有编码信息
sen_code = tokenizer.encode_plus("i like you")

In [44]:
tokenizer.convert_ids_to_tokens(sen_code['input_ids'])

['[CLS]', 'i', 'like', 'you', '[SEP]']

In [45]:
import torch
model.eval() # 将模型设为验证模式
input_ids = torch.tensor([sen_code['input_ids']]) # 添加batch维度并转化为tensor
token_type_ids = torch.tensor([sen_code['token_type_ids']])

In [46]:
# 将模型转化为eval模式
model.eval()
# 将模型和数据转移到cuda, 若无cuda,可更换为cpu
device = 'cuda'
tokens_tensor = input_ids.to(device)
segments_tensors = token_type_ids.to(device)
model.to(device)

# 进行编码
with torch.no_grad():
    # See the models docstrings for the detail of the inputs
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    # Transformers models always output tuples.
    # See the models docstrings for the detail of all the outputs
    # In our case, the first element is the hidden state of the last layer of the Bert model
    encoded_layers = outputs
# 得到最终的编码结果encoded_layers

In [47]:
a,b=encoded_layers

In [52]:
a.cpu().numpy()

array([[[ 0.01590374,  0.06605431, -0.09751298, ..., -0.2026468 ,
          0.07909803,  0.15123391],
        [ 0.0762463 , -0.26786408, -0.69057244, ...,  0.14479323,
          0.754324  ,  0.2614672 ],
        [ 0.2831399 , -0.13686614,  0.7657905 , ..., -0.24902762,
          0.39826664,  0.09002201],
        [-0.32801044, -1.0406164 , -0.04919878, ...,  0.2702459 ,
          0.46201882, -0.56252015],
        [ 0.7270662 , -0.00392812, -0.16390216, ...,  0.11239298,
         -0.5019356 , -0.4152661 ]]], dtype=float32)

In [49]:
b.shape

torch.Size([1, 768])

In [38]:
len(c)

13

In [39]:
len(d)

12

In [ ]:
class MyTask(torch.nn.Module):
    def __init__(self,feature_model_path=BERT_MODEL_PATH+'/bert-base-uncased-pytorch_model.bin'):
        super(MyTask,self).__init__()
        self.feature_ext=transformers.BertModel.from_pretrained(feature_model_path)
        

# QA任务下的 BEAT

In [8]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch
import transformers
MODEL_PATH = r"F:/workcode/FAQ/BERT_model"
# 实例化tokenizer
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH+'/bert-base-uncased-vocab.txt')

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [10]:
# 导入bert的model_config
model_config = transformers.BertConfig.from_pretrained(MODEL_PATH+'/bert-base-uncased-config.json')

In [12]:
# 首先新建bert_model
bert_model = transformers.BertModel.from_pretrained(MODEL_PATH+'/bert-base-uncased-pytorch_model.bin',config = model_config)
# 最终有两个输出，初始位置和结束位置（下面有解释）
model_config.num_labels = 2
# 同样根据bert的model_config新建BertForQuestionAnswering
model = BertForQuestionAnswering(model_config)
model.bert = bert_model

In [13]:
# 设定模式
model.eval()
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
# 获取input_ids编码
input_ids = tokenizer.encode(question, text)
# 手动进行token_type_ids编码，可用encode_plus代替
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
# 得到评分, 
start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
# 进行逆编码，得到原始的token 
all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
#['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'nice', 'puppet', '[SEP]']

In [16]:
# 对输出的答案进行解码的过程
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
# assert answer == "a nice puppet" 
# 这里因为没有经过微调，所以效果不是很好，输出结果不佳。
print(answer)
# 'was jim henson ? [SEP] jim henson was a nice puppet [SEP]'

jim henson ? [SEP] jim henson was a
